In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader
from joblib import dump, load
import os
import pickle
from joblib import load
import torch.nn as nn
from sklearn.base import BaseEstimator
import torch
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from matplotlib.lines import Line2D
import math 
import seaborn as sns
from scipy import stats
import sys
import warnings 

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.hidden_dim)
        attention_weights = nn.functional.softmax(attention_scores, dim=-1)
        attended_values = torch.matmul(attention_weights, value)

        return attended_values

class WellLoggingDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, num_layers, output_dim):
        super(LSTMRegressor, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, 1, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_dim1 * 2, hidden_dim2, 1, batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(hidden_dim2 * 2, hidden_dim3, 1, batch_first=True, bidirectional=True)

        self.fc = nn.Linear(hidden_dim3 * 2, output_dim)

    def forward(self, src):
        src = src.unsqueeze(1)

        lstm_output1, _ = self.lstm1(src)
        lstm_output2, _ = self.lstm2(lstm_output1)
        lstm_output3, _ = self.lstm3(lstm_output2)

        output = self.fc(lstm_output3[:, -1, :])

        return output



In [ ]:
# 在代码开始处添加
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
traindata = pd.read_csv('/Recon_Model-main/data/test.csv')
unique_layers = traindata['Layer'].unique()
targ_list = ['DTC']  
layer_feature = traindata['Layer']

onehot_encoder = OneHotEncoder(categories=[unique_layers], handle_unknown='ignore')

for targ in targ_list:
    print(f"Predicting for target variable: {targ}")
    all_features = ['DEPTH', 'GR', 'RD', 'RS', 'DEN', 'DTC']
    feature_names = [f for f in all_features if f != targ]
    target = traindata[targ]
    features = traindata[feature_names].values
    features = np.nan_to_num(features)
    target = np.nan_to_num(target)

    # Load saved model
    predictor = load(f'/BST_Model/model/model_{targ}.pkl')
    scalerX = load(f'/BST_Model/model/feature_scaler_{targ}.pkl')
    scalerY = load(f'/BST_Model/model/target_scaler_{targ}.pkl')
    onehot_encoder = load(f'/BST_Model/model/onehot_encoder_{targ}.pkl')

    # Use the loaded onehotencoder to encode the layer column independently
    layer_onehot = onehot_encoder.transform(layer_feature.values.reshape(-1, 1)).toarray() 
    
    # Combine the layer feature with other features
    features_encoded = np.concatenate((features, layer_onehot), axis=1)
    
    # Standardize features
    features_scaled = scalerX.transform(features_encoded)

    # Prediction using loaded models
    predictor.eval()
    with torch.no_grad():
        features_scaled_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
        target_pred_scaled = predictor(features_scaled_tensor)
        target_pred_scaled = target_pred_scaled.cpu().numpy()
    
    # Reverse scale predictor
    target_pred = scalerY.inverse_transform(target_pred_scaled.reshape(-1, 1)).flatten()

    test_rmse_original_scale = np.sqrt(mean_squared_error(target, target_pred))
    test_mae_original_scale = mean_absolute_error(target, target_pred)
    test_r2_original_scale = r2_score(target, target_pred)

    print(f"Final Test RMSE (Original Scale): {test_rmse_original_scale}")
    print(f"Final Test MAE (Original Scale): {test_mae_original_scale}")
    print(f"Final Test r2 (Original Scale): {test_r2_original_scale}")
   
    